# Realistic QKD Implementation

In this notebook you will implement a realistic quantum key distribution (QKD) protocol that produces a pair of matching one-time pads for Alice and Bob using the BB84 protocol.  Once Alice and Bob have a shared symmetric key, they can use it to send secret messages to one another.

* Look at each cell and examine the Python code.

* Fill in the missing code wherever you see **TODO: Put your code here.**

* Run each cell before going on to the next one.

Note that the realistic QKD implementation is very similar to the ideal QKD simulation but differs in several subtle but important ways.  The most obvious difference is that the Qubit class has been replaced by a Photon class.  Photons have similar **prepare** and **measure** methods as Qubits but take optional arguments.

The **prepare** method takes the optional argument **avgPhotonNumber**, which defines the intensity of Alice's light.  The default value is 1.0.  Higher values give Bob a better chance of detection but result in more multiple detections.

The **measure** method gives four possible outcomes instead of just two.  The additional two outcomes are "N" (no detections) and "M" (multiple detections).  Measurements are implemented with a polarizing beam splitter and two single-photon detectors.  If neither detector "clicks" an "N" outcome results.  If both detectors click an "M" outcome results.  A valid measurement is one in which only one of the two detectors clicks, indicating an "H" or "V" outcome (for **measureHV**) or a "D" or "A" outcome (for **measureDA**).

The **measure** method takes the optional argument **probDarkCount**, which defines the sensivity of Bob's detectors.  The default value is 0.1 (or 10%).  Higher values give fewer "N" outcomes but more dark counts.  Lower values give more "N" outcomes but fewer dark counts.  Dark counts are indistinguishable from valid detections but will result in greater mismatch between Alice and Bob's final one-time pads.





# The Photon Class

The cell below defines the **photon** class used in this notebook. You will need to click the Play button (the white triangle inside a black circle to the left) to run the cell and load the class. You can double click where it says SHOW CODE to see how the class is implemented and what are the available methods.  Click on the triple dots, then select "Form > Hide code" to hide the code again.





In [ ]:
#@title
import random
import numpy as np

class InputError(Exception):
    def __int__(self, expression, message):
        self.expression = expression
        self.message = message

class Photon:

    # default values
    AVG_PHOTON_NUMBER = 1.0
    PROB_DARK_COUNT = 0.1

    def __init__(self, Hcomp=0, Vcomp=0):
        self.alpha = Hcomp
        self.beta  = Vcomp

    # This is for debugging purposes only!
    def toString(self):
        if np.isreal(self.alpha):
            string = str(self.alpha) + "|H> "
        else:
            string = str(self.alpha) + "|H> "
        if np.isreal(self.beta):
            if self.beta >= 0:
                string += "+ " + str(self.beta) + "|V>"
            else:
                string += "- " + str(-self.beta) + "|V>"
        else:
            string += "+ " + str(self.beta) + "|V>"
        return string

    def prepareVacuum(self):
        energyPerMode = 0.5; # in units of hbar*omega
        x0 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        y0 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        x1 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        y1 = np.sqrt(energyPerMode)*random.gauss(0,1)/np.sqrt(2)
        self.alpha = complex(x0, y0)
        self.beta  = complex(x1, y1)

    def prepare(self, alpha, beta, avgPhotonNumber=AVG_PHOTON_NUMBER):
        if avgPhotonNumber < 0:
            raise InputError()
        vac = Photon()
        vac.prepareVacuum()
        self.alpha = alpha * np.sqrt(avgPhotonNumber) + vac.alpha
        self.beta  = beta  * np.sqrt(avgPhotonNumber) + vac.beta

    def prepareH(self, avgPhotonNumber=AVG_PHOTON_NUMBER):
        self.prepare(1, 0, avgPhotonNumber)

    def prepareV(self, avgPhotonNumber=AVG_PHOTON_NUMBER):
        self.prepare(0,1, avgPhotonNumber)

    def prepareD(self, avgPhotonNumber=AVG_PHOTON_NUMBER):
        self.prepare(1/np.sqrt(2),  1/np.sqrt(2), avgPhotonNumber)

    def prepareA(self, avgPhotonNumber=AVG_PHOTON_NUMBER):
        self.prepare(1/np.sqrt(2), -1/np.sqrt(2), avgPhotonNumber)

    def measureHV(self, probDarkCount=PROB_DARK_COUNT):
        if probDarkCount < 0 or probDarkCount > 1:
            raise InputError
        threshold  = -0.5*np.log(1 - np.sqrt(1-probDarkCount))
        intensityH = abs(self.alpha)**2
        intensityV = abs(self.beta)**2
        # The photon is absorbed by the detector:
        self.prepareVacuum()
        # The outcome is determined by threshold exceedances:
        if intensityH <= threshold and intensityV <= threshold:
            return "N" # no detection (invalid measurement)
        elif intensityH >  threshold and intensityV <= threshold:
            return "H" # single H photon detected
        elif intensityH <=  threshold and intensityV > threshold:
            return "V" # single V photon detected
        else:
            return "M" # multiple detections (invalid measurement)

    def measureDA(self, probDarkCount=PROB_DARK_COUNT):
        a = self.alpha
        b = self.beta
        self.alpha = (a+b)/np.sqrt(2)
        self.beta  = (a-b)/np.sqrt(2)
        outcome = self.measureHV(probDarkCount)
        if outcome == "H": return "D"
        if outcome == "V": return "A"
        else: return outcome

    def measureRL(self, probDarkCount):
        a = self.alpha
        b = self.beta
        self.alpha = (a-b*1j)/np.sqrt(2)
        self.beta  = (a+b*1j)/np.sqrt(2)
        outcome = self.measureHV(probDarkCount)
        if outcome == "H": return "R"
        if outcome == "V": return "L"
        else: return outcome

    def applyPolarizer(self, theta, phi):
	      # Apply a polarizing filter according to the input parameters.
	      # theta=0,    phi=0:     H polarizer
	      # theta=pi,   phi=0:     V polarizer
	      # theta=pi/2, phi=0:     D polarizer
	      # theta=pi/2, phi=pi:    A polarizer
	      # theta=pi/2, phi=pi/2:  R polarizer
	      # theta=pi/2, phi=-pi/2: L polarizer
        z = np.exp(1j*phi)
        a = self.alpha
        b = self.beta
        self.alpha = a*(1+np.cos(theta))/2 + b*np.sin(theta)/2*np.conj(z)
        self.beta  = a*np.sin(theta)/2*z + b*(1-np.cos(theta))/2
        # Now add an extra vacuum component.
        vac = photon.Photon()
        vac.prepareVacuum()
        a = vac.alpha
        b = vac.beta
        self.alpha = self.alpha + a*(1-np.cos(theta))/2 + b*(-np.sin(theta)/2)*np.conj(z)
        self.beta  = self.beta  + a*(-np.sin(theta)/2)*z + b*(1+np.cos(theta))/2

    def applyUnitaryGate(self, theta, phi, lamb):
        U = [[0,0],[0,0]]
        z1 = np.exp(1j*phi)
        z2 = -np.exp(1j*lamb)
        z3 = np.exp(1j*(lamb+phi))
        U[0][0] = np.cos(theta/2)
        U[1][0] = np.sin(theta/2)*z1
        U[0][1] = np.sin(theta/2)*z2
        U[1][1] = np.cos(theta/2)*z3
        a = self.alpha
        b = self.beta
        self.alpha = U[0][0]*a + U[0][1]*b
        self.beta  = U[1][0]*a + U[1][1]*b

    def applyXGate(self):
        # Applies the Pauli X gate
        self.applyUnitaryGate(np.pi, 0, np.pi)

    def applyYGate(self):
        # Applies the Pauli Y gate
        self.applyUnitaryGate(np.pi, np.pi/2, np.pi/2)

    def applyZGate(self):
        # Applies the Pauli X gate
        self.applyUnitaryGate(0, np.pi, 0)

    def applyHGate(self):
        # Applied the Hadamard (half-wavelength) gate
        self.applyUnitaryGate(np.pi/2, 0, np.pi)

    def applyQGate(self):
        # Applies the SH (quarter-wavelength) gate
        self.applyUnitaryGate(np.pi/2, np.pi/2, np.pi)

    def applyNoisyGate(self, p):
        # This operation acts as a depolarizing channel.
	      # p = 0 leaves the photon unchanged.
	      # p = 1 yields a completely random photon.
	      # 0 < p < 1 yields a partially random photon.
        if p < 0 or p > 1:
            raise InputError
        theta = np.arccos(1 - 2*random.uniform(0,1)*p)
        phi   = p*(2*random.uniform(0,1) - 1)*np.pi
        lamb  = p*(2*random.uniform(0,1) - 1)*np.pi
        self.applyUnitaryGate(theta, phi, lamb)

    def applyAttenuation(self, r):
      	# This operation acts as a partially reflecting beam splitter.
	      # r = 0 leaves the photon unchanged.
	      # r = 1 completely absorbs the photon, leaving a vacuum state.
	      # 0 < r < 1 partially attenuates the photon and adds some vacuum.
	      # r is the reflectivity.
        if r < 0 or r > 1:
            raise InputError
        t = np.sqrt(1-r*r) # t is the transmissivity.
        vac = photon.Photon()
        vac.prepareVacuum()
        self.alpha = (self.alpha)*t + (vac.alpha)*r
        self.beta  = (self.beta )*t + (vac.beta)*r


# Alice sends photons to Bob.

## Alice generates her raw key
Take a look at the code below and run the cell.  You'll be asked to write something similar in the next cell.

In [ ]:
MESSAGE_LENGTH = 10 # length of the text message we want to send (We can always add more later.)

n = 8  * MESSAGE_LENGTH # number of qubits  (We multiply by 8 because each character is represented by 8 bits in ASCII.)

rawKeyAlice = "" # start with an empty string

for i in range(n): # Iterate over the number of qubits (0 to n-1)
    # Append a random character ('0' or '1') to the end.
    rawKeyAlice += random.choice(['0','1'])

print("rawKeyAlice = " + rawKeyAlice) # print out Alice's raw key

# Sample Output:
# rawKeyAlice    = 01110111001011001100001011000100001101001110111000011100101111101000000010101110

## Alice chooses the encoding basis for each key bit.
Define a string of length *n* called **basisAlice** that consists of the characters '+' and 'x', each chosen randomly.  Here, '+' means Alices an H or V qubit, while 'x' means Alice prepares a D or A qubit.  This is exactly the same as the ideal case you did earlier.

In [ ]:
basisAlice = ""

# TODO: Put your code here.
for i in range(n):
    basisAlice += random.choice(['+','x'])
# -------------------------

print("basisAlice  = " + basisAlice)

# Sample Output:
# basisAlice  = xxx++x+x++++xx+x++xx++xxx+++x+xx++++x+x++xx++xxxxxx++++x+x+x++x+x++++xx+x+++xx++

## Alice selects a photon state according to the key and basis.

Define a string called **photonAlice** that is the same length as rawKeyAlice and basisAlice but is made up of the characters 'H', 'V', 'D', 'A'.  Each element of the string specifies the polarization state Alice will use, according to the following scheme:

* Use 'H' if they key is '0' and the basis is '+'.
* Use 'V' if they key is '1' and the basis is '+'.
* Use 'D' if they key is '0' and the basis is 'x'.
* Use 'A' if they key is '1' and the basis is 'x'.

This is similar to the ideal case, except now we're using the variable **photonAlice** instead of **qubitAlice**.

In [ ]:
photonAlice = ""

# TODO: Put your code here.
for i in range(n):
    if basisAlice[i]=='+':
        if rawKeyAlice[i]=='0':
            photonAlice += 'H'
        elif rawKeyAlice[i]=='1':
            photonAlice += 'V'
    elif basisAlice[i]=='x':
        if rawKeyAlice[i]=='0':
            photonAlice += 'D'
        elif rawKeyAlice[i]=='1':
            photonAlice += 'A'
# -------------------------

print("photonAlice  = " + photonAlice)

# Sample Output:
# photonAlice  = AADHHADDAVDAHAVADDAHVDDDHDVDAHVVAVDVHVDDAHAVHADDHAHDHVHADDHDDAVAVVVHDDHHHHDVHHHV

## Alice prepares and sends each photon.
Prepare each photon according to the chosen polarization state.

This is similar to what you did for the ideal case, except the Photon class prepare method has an optional argument (avgPhotonNumber).

In [ ]:
photonArray = [Photon() for i in range(n)] # define an array of qubit objects

# TODO: Put your code here.
for i in range(n):
    if   photonAlice[i]=='H':
      photonArray[i].prepareH()
    elif photonAlice[i]=='V':
      photonArray[i].prepareV()
    elif photonAlice[i]=='D':
      photonArray[i].prepareD()
    elif photonAlice[i]=='A':
      photonArray[i].prepareA()
    print(photonArray[i].toString()) # Feel free to comment this out if you like.
# -------------------------

# Eve intercepts and resends photons

In this section, an eavesdropper named Eve will intercept some of Alice's qubits and try to steal some of the secret key.

Your adversary group will put their code here, but feel free to put your own Eve code just to test out your version of Alice and Bob's protocol.

Eve is free to use any trick allowable by the Photon class.

# Bob measures the received photons.

## Bob selects which basis to measure in.
Define a string of length *n* called **basisBob** that consists of the characters '+' and 'x', each chosen randomly.  Here, '+' means Bob measures in the H/V basis, while 'x' means Bob measures in the D/A basis.  *Hint:* This is similar to basisAlice, but Bob's choices will be different from those of Alice.  This is also exactly the same as the ideal case you did earlier.

In [ ]:
basisBob = ""

# TODO: Put your code here.
for i in range(n):
    basisBob += random.choice(['+','x']) # '+'=H/V, 'x'=D/A
# -------------------------

print("basisBob    = " + basisBob)

# Sample Output:
# basisBob    = +x+x+xxxxxxxxxxx+++xx++xx+xxxxx+x++x+++++x+xxxx+x+xx++x+xx++++xx++xx+++x++x+xxxx

## Bob performs a measurement on each photon.
Use the methods **measureHV** and **measureDA** to perform a measurement on each qubit in the appropriate basis.  The outcome of each measurement will be one of the following characters: 'H', 'V', 'D', 'A', 'N' (no detections), 'M' (multiple detections).  Note that for the Photon class the measure method takes an optional argument (probDarkCount).

Using the measurement outcomes, define a string of length *n* called **outcomeBob** that consists of the measurement outcomes for each photon.

In [ ]:
outcomeBob = ""

# TODO: Put your code here.
for i in range(n):
    if basisBob[i]=='+':
        outcomeBob += photonArray[i].measureHV()
    elif basisBob[i]=='x':
        outcomeBob += photonArray[i].measureDA()
# -------------------------

print("outcomeBob  = " + outcomeBob)

# Sample Output:
# outcomeBob  = HMAVHNAHNAHNNNHDANNNNVDNNNDNNNNVNANNNANHNNHHMANNDNVNNNNNNNNNNNNNNNNVDAAHVNHANHVA

## Bob infers the raw key.
Use Bob's measurement outcomes to determine the raw key sent by Alice.  Specifcally, define a string of length *n* called **rawKeyBob** that consists of the characters '0' and '1', depending on each of Bob's measurement outcomes.  If the measurement is invalid (an 'N' or 'M' outcome), put a dash ('-') instead of a '0' or '1'.

In [ ]:
rawKeyBob = ""

# TODO: Put your code here.
for i in range(n):
    if outcomeBob[i]=='H' or outcomeBob[i]=='D':
        rawKeyBob += '0'
    elif outcomeBob[i]=='V' or outcomeBob[i]=='A':
        rawKeyBob += '1'
    else:
        rawKeyBob += '-'
# -------------------------

print("rawKeyBob = " + rawKeyBob)

# Sample Output:
# rawKeyBob = 0-110-10-10---001----10---0----1-1---1-0--00-1--0-1----------------101101-01-011

# Alice and Bob extract their shared secret key.

## Alice and Bob publicly announce which bases they chose.
After Bob completes his measurements, he and Alice announce publicly which bases they chose (either '+' = H/V or 'x' = D/A).  They still keep secret their raw keys, but now they know which parts should match and which might not.

Define two strings of length *n* called **siftedKeyAlice** and **siftedKeyBob** consisting of the characters '0', '1', and ' ' (a single space).  Put a single blank space wherever Alice and Bob's choice of basis do not match (or wherever Bob got an invalid outcome).  Do the same for wherever Bob's has an invalid measurement (indicated by a dash).  Wherever they *do* match, put the value of the raw key (either '0' or '1').  It is possible that these two keys do not match, even without Eve.

In [ ]:
siftedKeyAlice = ""
siftedKeyBob   = ""

# TODO: Put your code here.
for i in range(n):
    if basisAlice[i] == basisBob[i] and rawKeyBob[i] != '-':
        siftedKeyAlice += rawKeyAlice[i]
        siftedKeyBob   += rawKeyBob[i]
    else:
        siftedKeyAlice += ' '
        siftedKeyBob   += ' '
# -------------------------

print("siftedKeyAlice = " + siftedKeyAlice)
print("siftedKeyBob   = " + siftedKeyBob)

# Sample Output:
# siftedKeyAlice =      110 1         0      11               0  1             1      1   0
# siftedKeyBob   =      101 1         0      01               0  1             1      1   0

## Compare Alice and Bob's sifted keys.

Run this cell.  This computes the true quantum bit error rate (QBER), which is the percentage of Alice and Bob's sifted keys that match.  Note that blank spaces are not counted, as they are not really part of the sifted key.

Of course, Alice and Bob don't actually have access to this information.  It's just for you to check things for yourself.

In [ ]:
# initialize some variables
numMatch = 0
numDontMatch = 0

# compute the number of samples that match
for i in range(n):
    if siftedKeyAlice[i] != ' ' and siftedKeyBob[i] != ' ':
      if siftedKeyAlice[i] == siftedKeyBob[i]:
        numMatch += 1
      else:
        numDontMatch += 1

# compute the quantum bit error rate (QBER)
if numMatch + numDontMatch > 0:
    trueQBER = numDontMatch / ( numMatch + numDontMatch ) * 100
else:
    trueQBER = float('nan')

print("trueQBER = " + str(trueQBER) + "%")

# Sample Output:
# trueQBER = 8.25635103926097%

## Alice and Bob select a portion of the key to sample

Normally, Alice and Bob would keep their sifted keys secret.  Here, they will reveal some of their sifted key to check if they match.  A certain amount of mismatch is expected, due to the imperfections of realistic photons.  Too much may indicate an eavesdropper!

You will define a variable called **sampleIndex** that specifies which photons will be used (and later discarded) to check the security of the channel.  This is a string of length *n* consisting of '0's and '1's.  Here, '1' means "sample this sifted key element, and '0' means "don't".

In [ ]:
sampleIndex = ""

# TODO: Put your code here.
fracSampled = 0.1
for i in range(n):
    if siftedKeyAlice[i] != ' ' and random.uniform(0,1) < fracSampled:
        sampleIndex += '1'
    else:
        sampleIndex += '0'
# -------------------------

print("sampleIndex = " + sampleIndex)

## Bob determines if the quantum channel is secure

This cell computes the estimated quantum bit error rate (QBER), **estimatedQBER**.  This is the percentage of bits from the sampled keys that match.  If you have a good sample, it should be close to the true QBER, given by **trueQBER** above.

Finally, set the eavesdropper detection threshold, given by the variable THRESHOLD, to a suitable value between 0.0 and 1.0.  You want a number that's low enough to catch Eve but not so low that you get a bunch of false alerts.

Run this cell after choosing an appropriate value for THRESHOLD.



In [ ]:
# initialize some variables
numMatch = 0
numDontMatch = 0

# compute the number of samples that match
for i in range(n):
    if sampleIndex[i] == '1' and siftedKeyAlice[i] != ' ' and siftedKeyBob[i] != ' ':
      if siftedKeyAlice[i] == siftedKeyBob[i]:
        numMatch += 1
      else:
        numDontMatch += 1

# compute the estimated quantum bit error rate (QBER)
if numMatch + numDontMatch > 0:
    estimatedQBER = numDontMatch / ( numMatch + numDontMatch ) * 100
else:
    estimatedQBER = float('nan')

print("estimatedQBER = " + str(estimatedQBER) + "%")

# TODO:  Set THRESHOLD to an value between 0 and 100.
THRESHOLD = 10
# -------------------------
if estimatedQBER > THRESHOLD:
  channelSecure = False
  print("ALERT!  The quantum channel may not be secure.")
else:
  channelSecure = True
  print("All clear!  The quantum channel appears to be secure.")

## Alice and Bob discard the sampled portions

Create two new strings of length *n* called **secureAlice** and **secureBob**.  These are identical to **siftedAlice** and **siftedBob**, respectively, except you will replace the sampled keys with an blank space (' '), indicating their removal.

In [ ]:
secureKeyAlice = ""
secureKeyBob = ""

# TODO: Put your code here.
for i in range(n):
  if sampleIndex[i] == '1':
    secureKeyAlice += ' '
    secureKeyBob   += ' '
  else:
    secureKeyAlice += siftedKeyAlice[i]
    secureKeyBob   += siftedKeyBob[i]
# -------------------------

print("secureKeyAlice = " + secureKeyAlice)
print("secureKeyBob   = " + secureKeyBob)

## Alice and Bob store local copies of their one-time pad
Run this cell.  It will generate two files: **onetimepadAlice.txt** and **onetimepadBob.txt**.  These will be text files containing (hopefully!) identical strings of zeros and ones.  Alice and Bob can now use their local copies to send and receive secret, encrypted messages!

*Note:* The first time you run this cell, you'll be asked to log into your Google account and enter an authorization code.  This will allow you to read and write to your Colab Notebooks folder.

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

GOOGLE_DRIVE_DIR = "/content/drive/My Drive/"

with open(GOOGLE_DRIVE_DIR + "onetimepadAlice.txt", 'w') as fileA:
  for i in range(n):
    if siftedKeyAlice[i]=='0' or siftedKeyAlice[i]=='1':
      fileA.write(siftedKeyAlice[i])

with open(GOOGLE_DRIVE_DIR + "onetimepadBob.txt", 'w') as fileB:
  for i in range(n):
    if siftedKeyBob[i]=='0' or siftedKeyBob[i]=='1':
      fileB.write(siftedKeyBob[i])

with open(GOOGLE_DRIVE_DIR + "onetimepadAlice.txt", 'r') as fileA:
  print(fileA.read())

with open(GOOGLE_DRIVE_DIR + "onetimepadBob.txt", 'r') as fileB:
  print(fileB.read())

drive.flush_and_unmount()

# Sample Output:
# Mounted at /content/drive
# 01101111011001110011101101011011000111110
# 01101111011001110011101101011011000111110